# Edge Device Federated Learning

In [ ]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import make_classification
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

## Create simulated medical diagnostic data

In [ ]:
def generate_site_data(seed):
    X, y = make_classification(n_samples=200, 
                               n_features=10, 
                               n_informative=6,
                               n_redundant=2, n_classes=2, 
                               random_state=seed)
    return train_test_split(X, y, 
                            test_size=0.3, 
                            random_state=seed + 10)

## Simulate data for three different hospital sites

In [ ]:
# Simulate data for 3 NHS sites
site_data = [generate_site_data(seed) for seed in [1, 2, 3]]

## Fit a model per site

In [ ]:
local_models = []
for X_train, X_test, y_train, y_test in site_data:
    model = LogisticRegression(max_iter=500).fit(X_train, y_train)
    local_models.append((model, X_test, y_test))

## Evaluate local models per site

In [ ]:
print("AUC for individual NHS sites:")
for i, (model, X_test, y_test) in enumerate(local_models):
    auc = roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])
    print(f"Hospital Site {i+1}: AUC = {auc:.3f}")

## Create federated average 

In [ ]:
def federated_average(models):
    coefs = np.mean([m.coef_ for m, _, _ in models], 
                    axis=0)
    intercepts = np.mean([m.intercept_ for m, _, _ in models], 
                         axis=0)
    global_model = LogisticRegression()
    global_model.coef_ = coefs
    global_model.intercept_ = intercepts
    global_model.classes_ = np.array([0, 1])
    return global_model

## Federated Model - Globally in the Cloud

In [ ]:
for site in [2, 3]:
    selected_models = local_models[:site]
    global_model = federated_average(selected_models)
    X_combined = np.vstack([X for _, X, _ in selected_models])
    y_combined = np.hstack([y for _, _, y in selected_models])
    y_pred_prob = global_model.predict_proba(X_combined)[:, 1]
    auc = roc_auc_score(y_combined, y_pred_prob)
    print(f"\nGlobal Model (aggregated over {site} site(s)) — AUC: {auc:.3f}")